In [13]:
import sys
import json
from pathlib import Path
from google.oauth2.credentials import Credentials

root_dir = Path("/workspaces/DeliveryTracker")

sys.path.insert(0, str(root_dir))


credential_json = json.load((root_dir / "google_token/logged_in_credentials.json").open())
credentials = Credentials.from_authorized_user_info(credential_json)

In [14]:
from shared.GoogleServices import GmailService

In [16]:
gs = GmailService(credentials)
gs.authenticate()

In [23]:
from datetime import datetime, timedelta, timezone


time_threshold = datetime.now(timezone.utc) - timedelta(days=7)
query = f"from:no-reply@winsim.de subject:'Ihre winSIM-Rechnung' after:{int(time_threshold.timestamp())}"
    
    # 2. Use the Gmail API to find message IDs that match the query.
response = gs.service.users().messages().list(userId='me', q=query).execute()
messages = response.get('messages', [])

In [24]:
messages

[{'id': '1945b4defe14256d', 'threadId': '1945b4defe14256d'}]